In [15]:
import pandas as pd
from pathlib import Path
import os
import shutil

#Move downloaded files to the working directory
source = Path('/Users/Kuba/Downloads')              
dest = Path('/Users/Kuba/Desktop/ScoutLog')

files = source.glob('*table-data*')

for f in files:
    shutil.move(f, dest)


#Load data from .csv files (order of downloads from left to right)
tab1 = pd.read_csv('table-data.csv')
tab2 = pd.read_csv('table-data (1).csv')
tab3 = pd.read_csv('table-data (2).csv')
tab4 = pd.read_csv('table-data (3).csv')
tab5 = pd.read_csv('table-data (4).csv')



#Rename 'Hot Contracts' column names
tab5.rename(columns = {
    'Network': 'chain',
    'Address': 'address',
}, inplace = True)


data_new = pd.concat([tab1, tab2, tab3, tab4]).reset_index()

#Drop unnecessary columns and join for final new data
data_new.drop(data_new.columns[3:14], axis=1, inplace = True)
data_new.drop('index', axis=1, inplace = True)
tab5.drop(tab5.columns[2:13], axis = 1, inplace = True)

data_new = pd.concat([data_new, tab5]).reset_index()

#Have to drop index once again but dont know why?
data_new.drop('index', axis = 1, inplace = True)

#Clean address column
data_new['address'] = data_new['address'].str.strip('</a>').str.strip("<a href=").str[:-58].str.strip("'")


#If 'data' is present in the directory, find new addresses
if os.path.exists('data.csv') is True:
    #Read 'data' file
    data = pd.read_csv('data.csv', index_col=[0])
    #Join data and new data, drop duplicate rows, export rest to Excel
    to_check = pd.merge(data_new, data, on = 'address', how = 'left').reset_index()
    to_check = data_new.loc[~data_new['address'].isin(data['address'])]

    #Export results to Excel file in the working directory
    to_check.to_excel('to_check.xlsx', index = False)
    

#Update the data file thats being compared against
data_new.to_csv('data.csv')

#Remove the 'table-data' files from working directory
files_remove = dest.glob('*table-data*')
for f in files_remove:
    os.remove(f)
